In [35]:
import pandas as pd
import time, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
input_file = "test.csv"
output_file = "returndata.csv"
max_retries = 3
delay_between = 2
df = pd.read_csv(input_file)
for col in [
    "MOT Status", "Road Tax Status", "Road Tax Expiry", "Days Left",
    "12 Months Cost", "6 Months Cost", "CO2 Output","Body Style"
]:
    if col not in df.columns:
        df[col] = ""


if os.path.exists(output_file):
    print(f"🔁 Resuming from existing file: {output_file}")
    df_old = pd.read_csv(output_file)
    for col in df.columns:
        if col in df_old.columns:
            df[col] = df_old[col]



options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def login_totalcarcheck(email, password):
    login_url = "https://totalcarcheck.co.uk/Account/Login"
    driver.get(login_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "UserName"))
    )


    driver.find_element(By.ID, "UserName").send_keys(email)
    driver.find_element(By.ID, "Password").send_keys(password)
    driver.find_element(By.CSS_SELECTOR, "input.btn.btn-primary").click()

    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "userIdLink"))
    )

    print("✅ Logged in successfully! User account element found.")

email = "sultanmirza0501@icloud.com"
password = "Muhssan7865"
login_totalcarcheck(email, password)


def fetch_vehicle_info(reg):

    for attempt in range(1, max_retries + 1):
      
        try:
            url = f"https://totalcarcheck.co.uk/FreeCheck?regno={reg.replace(' ', '+')}"
            driver.get(url)


            mot_element = WebDriverWait(driver, 12).until(
                EC.presence_of_element_located((By.XPATH, '//span[text()="MOT Status"]/following::span[1]'))
            )
            mot_status = mot_element.text.strip()


            tax_element = WebDriverWait(driver, 12).until(
                EC.presence_of_element_located((By.XPATH, '//span[text()="Road Tax Status"]/following::span[1]'))
            )
            tax_status = tax_element.text.strip()


            try:
                days_left = driver.find_element(By.XPATH, '//span[text()="Days Left"]/following::span[1]').text.strip()
            except:
                days_left = ""


            tax_expiry = tax_status.replace("Expires:", "").replace("Expired:", "").strip()


            try:
                cost_12 = driver.find_element(By.XPATH, '//span[text()="12 Months Cost"]/following::span[1]').text.strip()
            except:
                cost_12 = ""


            try:
                cost_6 = driver.find_element(By.XPATH, '//span[text()="6 Months Cost"]/following::span[1]').text.strip()
            except:
                cost_6 = ""

            try:
                co2_output = driver.find_element(By.XPATH, '//span[text()="CO₂ Output"]/following::span[1]').text.strip()
            except:
                co2_output = ""
            
            try:
                body_style = driver.find_element(By.XPATH,'//span[text()="Body Style"]/following::span[1]').text.strip()
            except:
                body_style = ""
            data = {}   
            try:
                rows = driver.find_elements(By.XPATH, '//table[@class="table table-responsive table-freecheck"]//tr')
                for row in rows:
                    try:
                        label = row.find_element(By.XPATH, './/span[@class="cert-label"]').text.strip()
                        value = row.find_element(By.XPATH, './/span[contains(@class,"cert-data")]').text.strip()
                        data[label] = value
                    except Exception:
                        continue
            except Exception as e:
                print(f"⚠️ Error while extracting table: {e}")

            
                

            print(f"✅ {reg}: MOT={mot_status} | TAX={tax_status} | Days={days_left} | 12M={cost_12} | 6M={cost_6} | CO₂={co2_output} | body_style={body_style}")
            return mot_status, tax_status, tax_expiry, days_left, cost_12, cost_6, co2_output,body_style,data

        except Exception as e:
            print(f"⚠️ Attempt {attempt} failed for {reg}: {e}")
            time.sleep(3)


    return "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A","N/A", {}



for index, row in df.iterrows():
    reg = str(row["Reg"]).strip()
    if not reg or reg.lower() == "nan":
        continue

    if (
        str(df.at[index, "MOT Status"]).strip() not in ["", "N/A"]
        and str(df.at[index, "Road Tax Status"]).strip() not in ["", "N/A"]
    ):
        print(f"⏭️ Skipping already done: {reg}")
        continue


    mot_status, tax_status, tax_expiry, days_left, cost_12, cost_6, co2_output, body_style, data = fetch_vehicle_info(reg)


    if str(df.at[index, "MOT Status"]).strip() in ["", "N/A"]:
        df.at[index, "MOT Status"] = mot_status
    if str(df.at[index, "Road Tax Status"]).strip() in ["", "N/A"]:
        df.at[index, "Road Tax Status"] = tax_status
    if str(df.at[index, "Road Tax Expiry"]).strip() in ["", "N/A"]:
        df.at[index, "Road Tax Expiry"] = tax_expiry
    if str(df.at[index, "Days Left"]).strip() in ["", "N/A"]:
        df.at[index, "Days Left"] = days_left
    if str(df.at[index, "12 Months Cost"]).strip() in ["", "N/A"]:
        df.at[index, "12 Months Cost"] = cost_12
    if str(df.at[index, "6 Months Cost"]).strip() in ["", "N/A"]:
        df.at[index, "6 Months Cost"] = cost_6
    if str(df.at[index, "CO2 Output"]).strip() in ["", "N/A"]:
        df.at[index, "CO2 Output"] = co2_output
    if str(df.at[index, "Body Style"]).strip() in ["", "N/A"]:
        df.at[index, "Body Style"] = body_style


    for key, value in data.items():
        if key in df.columns:
            print(key)
            print(value)
            if str(df.at[index, key]).strip() in ["", "N/A","na"," "]:
                df.at[index, key] = value


    df.to_csv(output_file, index=False)
    time.sleep(delay_between)

driver.quit()
print(f"\n✅ All done! Data saved to {output_file}")



✅ Logged in successfully! User account element found.
✅ LS70 NLV: MOT=Expired: 27 Oct 2025 | TAX=Expires: 01 Oct 2026 | Days=324 days left | 12M=£195 | 6M=£107.25 | CO₂=132 g/km (H) | body_style=Hatchback
MOT Status
Expired: 27 Oct 2025
Road Tax Status
Expires: 01 Oct 2026
Days Left
324 days left
Model
A3 Technik 35 Tfsi
Colour
Black
Body Style
Hatchback
Fuel Type
Petrol
Vehicle Age
5 years 0 months
12 Months Cost
£195
6 Months Cost
£107.25
✅ RO71 GZM: MOT=Expires: 06 Oct 2026 | TAX=Expired: 20 Oct 2025 | Days=329 days left | 12M=£195 | 6M=£107.25 | CO₂=120 g/km (G) | body_style=Saloon
MOT Status
Expires: 06 Oct 2026
Days Left
329 days left
Road Tax Status
Expired: 20 Oct 2025
Model
A3 S Line 30 Tfsi Mhev S-A
Colour
Black
Body Style
Saloon
Fuel Type
Petrol
Vehicle Age
4 years 1 month
12 Months Cost
£195
6 Months Cost
£107.25
✅ HJ22 GKN: MOT=Expires: 20 Mar 2026 | TAX=Expired: 13 Oct 2025 | Days=129 days left | 12M=£195 | 6M=£107.25 | CO₂=0 g/km (A) | body_style=SUV
MOT Status
Expires: 